# Analysis of large data sets: Geo-Python

## A (short) introduction to Iris

### What is Iris?

 * Iris is a Python library written by a team at the Met Office to make working with model data easier
 * It has been evolving over the last 4 or 5 years and is now at version 1.10
 * Lots of ongoing development and there is good support available

### Why use Iris?

 * It keeps all of your arrays together in one place - the data, the coordinates etc.
 * It takes care of metadata - and keeps it CF-conformant
 * It takes care of the layout of your data (which dimensions are which)
 * Can load pp and GRIB files as well as NetCDF
 * Nice plotting routines

## Iris and the cube

The top level object in Iris is called a cube. A cube contains data and metadata about a single phenomenon and is an implementation of the data model interpreted from the *Climate and Forecast (CF) Metadata Conventions*.

Each cube has:

 * A data array (typically a NumPy array).
 * A "name", preferably a CF "standard name" to describe the phenomenon that the cube represents.
 * A collection of coordinates to describe each of the dimensions of the data array. These coordinates are split into two types:
    * Dimensioned coordinates are numeric, monotonic and represent a single dimension of the data array. There may be only one dimensioned coordinate per data dimension.
    * Auxilliary coordinates can be of any type, including discrete values such as strings, and may represent more than one data dimension.

A fuller explanation is available in the [Iris user guide](http://scitools.org.uk/iris/docs/latest/userguide/iris_cubes.html).

Let's take a simple example to demonstrate the cube concept.

Suppose we have a ``(3, 2, 4)`` NumPy array:

![](../images/multi_array.png)


Where dimensions 0, 1, and 2 have lengths 3, 2 and 4 respectively.

The Iris cube to represent this data may consist of:

 * a standard name of "air_temperature" and units of "kelvin"

 * a data array of shape ``(3, 2, 4)``

 * a coordinate, mapping to dimension 0, consisting of:
     * a standard name of "height" and units of "meters"
     * an array of length 3 representing the 3 height points
     
 * a coordinate, mapping to dimension 1, consisting of:
     * a standard name of "latitude" and units of "degrees"
     * an array of length 2 representing the 2 latitude points
     * a coordinate system such that the latitude points could be fully located on the globe
     
 * a coordinate, mapping to dimension 2, consisting of:
     * a standard name of "longitude" and units of "degrees"
     * an array of length 4 representing the 4 longitude points
     * a coordinate system such that the longitude points could be fully located on the globe

Pictorially the cube has taken on more information than a simple array:

![](../images/multi_array_to_cube.png)

## Working with a cube

In [1]:
import iris
import numpy as np

In [2]:
print(iris.__version__)
print(np.__version__)

1.10.0
1.11.2


Whilst it is possible to construct a cube by hand, a far more common approach to getting hold of a cube is to use the Iris load function to access data that already exists in a file.

In [3]:
fname = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(fname)
print(cubes)

0: air_potential_temperature / (K)     (time: 3; model_level_number: 7; grid_latitude: 204; grid_longitude: 187)
1: surface_altitude / (m)              (grid_latitude: 204; grid_longitude: 187)


We can see that we've loaded two cubes, one representing the "surface_altitude" and the other representing "air_potential_temperature". We can infer even more detail from this printout; for example, what are the dimensions and shape of the "air_potential_temperature" cube?

Above we've printed the ``iris.cube.CubeList`` instance representing all of the cubes found in the given filename. However, we can see more detail by printing individual cubes:

In [6]:
air_pot_temp = cubes[0]
print(air_pot_temp)

air_potential_temperature / (K)     (time: 3; model_level_number: 7; grid_latitude: 204; grid_longitude: 187)
     Dimension coordinates:
          time                           x                      -                 -                    -
          model_level_number             -                      x                 -                    -
          grid_latitude                  -                      -                 x                    -
          grid_longitude                 -                      -                 -                    x
     Auxiliary coordinates:
          forecast_period                x                      -                 -                    -
          level_height                   -                      x                 -                    -
          sigma                          -                      x                 -                    -
          surface_altitude               -                      -                 x                

We can dig even deeper and print individual coordinates:

In [5]:
print(air_pot_temp.coord('model_level_number'))

DimCoord(array([ 1,  4,  7, 10, 13, 16, 19], dtype=int32), standard_name='model_level_number', units=Unit('1'), attributes={'positive': 'up'})


## Cube attributes

In [ ]:
cube = iris.load_cube(iris.sample_data_path('A1B_north_america.nc'))
print(cube)

To access a cube's data array the ``data`` property exists. This is either a NumPy array or in some cases a NumPy masked array. 

It is important to note that for most of the supported filetypes in Iris, the cube's data isn't actually loaded until you request it via this property (either directly or indirectly). After you've accessed the data once, it is stored on the cube and thus won't be loaded from disk again.

To find the shape of a cube's data it is possible to call ``cube.data.shape`` or ``cube.data.ndim``, but this will trigger any unloaded data to be loaded. Therefore ``shape`` and ``ndim`` are properties available directly on the cube that do not unnecessarily load data.

In [ ]:
print(cube.shape)
print(cube.ndim)
print(type(cube.data))

The ``standard_name``, ``long_name`` and to an extent ``var_name`` are all attributes to describe the phenomenon that the cube represents. The ``name()`` method is a convenience that returns the first non-empty attributes in the order they are listed above. 

In [ ]:
print(cube.standard_name)
print(cube.long_name)
print(cube.var_name)
print(cube.name())

To rename a cube, it is possible to set the attributes manually, but it is generally easier to use the ``rename()`` method.

In [ ]:
cube.rename("A name that isn't a valid CF standard name")

In [ ]:
print(cube.standard_name)
print(cube.long_name)
print(cube.var_name)
print(cube.name())

The ``units`` attribute on a cube tells us the units of the numbers held in the data array. We can manually change the units, or better, we can convert the cube to another unit using the ``convert_units`` method, which will automatically update the data array.

In [ ]:
print(cube.units)
print(cube.data.max())
cube.convert_units('Celsius')
print(cube.units)
print(cube.data.max())

A cube has a dictionary for extra general purpose attributes, which can be accessed with the ``cube.attributes`` attribute:

In [ ]:
print(cube.attributes)
print(cube.attributes['STASH'])

## Coordinates

As we've seen, cubes need coordinate information to help us describe the underlying phenomenon. Typically a cube's coordinates are accessed with the ``coords`` or ``coord`` methods. The latter *must* return exactly one coordinate for the given parameter filters, where the former returns a list of matching coordinates, possibly of length 0.

For example, to access the time coordinate, and print the first 4 times:

In [ ]:
time = cube.coord('time')
print(time[:4])

The coordinate interface is very similar to that of a cube. The attributes that exist on both cubes and coordinates are: ``standard_name``, ``long_name``, ``var_name``, ``units``, ``attributes`` and ``shape``. Similarly, the ``name()``, ``rename()`` and ``convert_units()`` methods also exist on a coordinate.

A coordinate does not have ``data``, instead it has ``points`` and ``bounds`` (``bounds`` may be ``None``). In Iris, time coordinates are currently represented as "a number since an epoch":

In [ ]:
print(repr(time.units))
print(time.points[:4])
print(time.bounds[:4])

These numbers can be converted to datetime objects with the unit's ``num2date`` method. Dates can be converted back again with the ``date2num`` method:

In [ ]:
import datetime

print(time.units.num2date(time.points[:4]))
print(time.units.date2num(datetime.datetime(1970, 2, 1)))

Another important attribute on a coordinate is its coordinate system. Coordinate systems may be ``None`` for trivial coordinates, but particularly for spatial coordinates, they may be complex definitions of things such as the projection, ellipse and/or datum.

In [ ]:
lat = cube.coord('latitude')
print(lat.coord_system)

In this case, the latitude's coordinate system is a simple geographic latitude on a spherical globe of radius 6371229 (meters).

Sometimes it is desirable to add bounds to a coordinate that doesn't have any. 

The ``guess_bounds`` method on a coordinate is useful in this regard. 

For example, the latitude coordinate previously obtained does not have bounds, but we can either set some manually, or use the ``guess_bounds`` method:

In [ ]:
print(lat.points[:4])
print(lat.bounds)
if lat.bounds is None:
    lat.guess_bounds()
print(lat.bounds[:4])

### Exercise 1

1\. Using the file in ``iris.sample_data_path('atlantic_profiles.nc')`` load the data and print the cube list. Store these cubes in a variable called cubes.

2\. Loop through each of the cubes (e.g. ``for cube in cubes``) and print the standard name of each.

3\. Extract the "sea_water_potential_temperature" cube. Print the minimum, maximum, mean and standard deviation of the cube's data.

4\. Print the attributes of the cube.

5\. Print the names of all coordinates on the cube. (Hint: Remember the cube.coords method without any keywords will give us all of the cube's coordinates)

6\. Get hold of the "latitude" coordinate on the cube. Identify whether this coordinate has bounds. Print the minimum and maximum latitude points in the cube.

## Loading data into Iris

We've already seen the basic ``load`` function, but we can also control which cubes are actually loaded with *constraints*. The simplest constraint is just a string, which filters cubes based on their name:

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
print(iris.load(fname, 'air_potential_temperature'))

#### Note on sample_data_path:

Throughout this course we will make use of the sample data that Iris provides. The function ``iris.sample_data_path`` returns the appropriate path to the file in the Iris sample data collection. A common mistake for Iris users is to use the ``sample_data_path`` function to access data that is not part of Iris's sample data collection - this is bad practice and is unlikely to work in the future.

### Exercise 2

Print the result of ``iris.sample_data_path('uk_hires.pp')`` to verify that it returns a string pointing to a file on your system. Use this string directly in the call to ``iris.load`` and confirm the result is the same as in the previous example e.g.:

    print iris.load('/path/to/iris/sampledata/uk_hires.pp', 'air_potential_temperature')

### The three load functions: load, load_cube and load_cubes

There are three main load functions in Iris: ``load``, ``load_cube`` and ``load_cubes``.

1. **load** is a general purpose loading function. Typically this is where all data analysis will start, before more loading is refined with the more controlled loading from the other two functions.
2. **load_cube** returns a single cube from the given source(s) and constraint. There will be exactly one cube, or an exception will be raised.
3. **load_cubes** returns a list of cubes from the given sources(s) and constraint(s). There will be exactly one cube per constraint, or an exception will be raised.


Note: ``load_cube`` is a special case of ``load_cubes``, which can be seen with:

In [ ]:
c1, = iris.load(fname, 'surface_altitude')
c2 = iris.load_cube(fname, 'surface_altitude')
c3, = iris.load_cubes(fname, 'surface_altitude')
c1 == c2 == c3

In general, it is a good idea to make use of the ``load_cube``/``load_cubes`` functions rather than the generic ``load`` function in non-exploratory code. Doing so makes your code more resilient to changes in the data source, often results in more readable/maintainable code, and in combination with well defined constraints, often leads to improve load performance.

The load functions all accept a list of filenames to load, and any of the filenames can be "glob" patterns (http://docs.python.org/2/library/glob.html).

### Exercise 2 (continued)

Read in the files found at **``iris.sample_data_path('GloSea4', 'ensemble_010.pp')``** and
**``iris.sample_data_path('GloSea4', 'ensemble_011.pp')``** using a single load call. Do this by:

1\. providing a list of the two filenames.

2\. providing a suitable glob pattern. (Notice that **``iris.load(iris.sample_data_path('GloSea4', 'ensemble_01*.pp'))``** picks up too many files.)

## Saving cubes

The ``iris.save`` function provides a convenient interface to save Cube and CubeList instances.

To save some cubes to a NetCDF file:

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(fname)
iris.save(cubes, 'saved_cubes.nc')

In [ ]:
!ncdump -h saved_cubes.nc | head -n 20
!rm saved_cubes.nc

Extra keywords can be passed to specific fileformat savers.

**Task:** Go to the Iris reference documentation for ``iris.save``. What fileformats can Iris currently save to? What keywords are accepted to ``iris.save`` when saving a PP file?

## Indexing

Cubes can be indexed in a familiar manner to that of NumPy arrays:

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname, 'air_potential_temperature')
print(cube.summary(shorten=True))

In [ ]:
subcube = cube[..., ::2, 15:35, :10]
subcube.summary(shorten=True)

Note: the result of indexing a cube is *always* a copy and never a *view* on the original data.

### Concatenate ###

We have seen that merge combines a list of cubes with a common scalar coordinate to produce a single cube with a new dimension created from these scalar values.

But what happens if you try to combine cubes along a common dimension?

In [ ]:
fname = iris.sample_data_path('A1B_north_america.nc')
cube = iris.load_cube(fname)

cube_1 = cube[:10]
cube_2 = cube[10:20]
cubes = iris.cube.CubeList([cube_1, cube_2])
print(cubes)

These cubes should be able to be merged; after all, they have both come from the same original cube!

In [ ]:
print(cubes.merge())

Merge cannot be used to combine common non-scalar coordinates. Instead we must use concatenate, which joins together ("concatenates") common non-scalar coordinates to produce a single cube with the common dimension extended:

In [ ]:
print(cubes.concatenate())

As with merge, Iris contains functionality to simplify the identification process for causes of failed concatenations. The ``concatenate_cube`` method of a CubeList expects the list of cubes to contain only cubes that can be concatenated to produce a single cube. If they do not concatenate to a single cube, a descriptive error will be raised. For instance:

```
    >>> print cubes.concatenate_cube()
    Traceback (most recent call last):
      ...
    iris.exceptions.ConcatenateError: failed to concatenate into a single cube.
      Scalar coordinates differ: forecast_reference_time, height != forecast_reference_time
```

## Iteration

We can loop through all desired subcubes in a larger cube using the ``slices`` and ``slices_over`` methods.

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname,
                      iris.Constraint('air_potential_temperature',
                                      model_level_number=1))
print(cube.summary(True))

The ``slices`` method returns all the slices of a cube on the dimensions specified by the coordinates passed to the slices method.

So in this example, each grid_latitude / grid_longitude slice of the cube is returned:

In [ ]:
for subcube in cube.slices(['grid_latitude', 'grid_longitude']):
    print(subcube.summary(shorten=True))

The ``iris.iterate.izip`` function extends this concept and allows us to loop through multiple cubes at the same time:

In [ ]:
from iris.iterate import izip

iris.FUTURE.cell_datetime_objects = True

e1 = iris.load_cube(iris.sample_data_path('E1_north_america.nc'))
a1b = iris.load_cube(iris.sample_data_path('A1B_north_america.nc'))

for e1_slice, a1b_slice in izip(e1, a1b, coords=['latitude', 'longitude']):
    print(e1_slice.summary(True), e1_slice.coord('time').cell(0).point)
    print(a1b_slice.summary(True), a1b_slice.coord('time').cell(0).point)
    break

In this example, one real use for this functionality would be to plot the ``e1`` cube next to the ``a1b`` cube for each timestep.

We can use ``slices_over`` to return one subcube for each coordinate value in a specified coordinate. This helps us when trying to retrieve all the slices along a given cube dimension.

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname, 'air_potential_temperature')
for subcube in cube.slices_over('time'):
    print(subcube.summary(shorten=True))

## Cube maths

Basic mathematical operators exist on the cube to allow one to add, subtract, divide, multiply and perform other mathematical operations on cubes of a similar shape to one another:

In [ ]:
a1b = iris.load_cube(iris.sample_data_path('A1B_north_america.nc'))
e1 = iris.load_cube(iris.sample_data_path('E1_north_america.nc'))

print(e1.summary(True))
print(a1b)

In [ ]:
scenario_difference = a1b - e1
print(scenario_difference)

Notice that the resultant cube's name is now unknown and that the coordinates “time” and “forecast_period” have been removed; this is because these coordinates differed between the two input cubes.

It is also possible to operate on cubes with numeric scalars, NumPy arrays and even cube coordinates:

In [ ]:
e1 * e1.coord('latitude')

Cube broadcasting is also taking place, meaning that the two cubes don't need to have the same shape:

In [ ]:
print(e1 - e1.collapsed('time', iris.analysis.MEAN))

Sometimes operations don't exist in Iris, so it is important that we still have the power to update the cube's data directly. Whenever we do this though, we should be mindful of updating the necessary metadata on the cube:

In [ ]:
e1_hot = e1.copy()

e1_hot.data = np.ma.masked_less_equal(e1_hot.data, 280)
e1_hot.rename('air temperatures greater than 280K')

## Creating extra annotation coordinates for statistical convenience

Sometimes we want to be able to categorise data before performing statistical operations on it. For example, we might want to categorise our data by "daylight maximum" or "seasonal mean" etc. Both of these categorisations would be based on the time coordinate.

The ``iris.coord_categorisation`` module provides convenience functions to add some common categorical coordinates, and provides a generalised function to allow each creation of custom categorisations. 

In [ ]:
import iris.coord_categorisation as coord_cat

filename = iris.sample_data_path('ostia_monthly.nc')
cube = iris.load_cube(filename, 'surface_temperature')

The cube loaded represents the monthly air_temperature from April 2006 through to October 2010. Let's add a categorisation coordinate to this cube to identify the climatological season (i.e "djf", "mam", "jja" or "son") of each time point:

In [ ]:
coord_cat.add_season(cube, 'time', name='clim_season')

We can now use the cube's ``aggregated_by`` method to "group by and aggregate" on the season, to produce the seasonal mean:

In [ ]:
seasonal_mean = cube.aggregated_by('clim_season', iris.analysis.MEAN)

We can take this further by extracting by our newly created coordinate, producing a plot of the winter zonal mean:

In [ ]:
winter = seasonal_mean.extract(iris.Constraint(clim_season='djf'))

qplt.plot(winter.collapsed('latitude', iris.analysis.MEAN))
plt.title('Winter zonal mean surface temperature at $\pm5^{\circ}$ latitude')
plt.show()

### Custom categorisation ###

Custom categorisation can be achieved with an arbitrary function. For example, the existing ``add_year`` categorisor takes the 'time' coordinate, and creates a 'year' coordinate. This could be achieved without using the available ``add_year`` by:

In [ ]:
def year_from_time(coord, point):
    return coord.units.num2date(point).year

coord_cat.add_categorised_coord(cube, 'year', cube.coord('time'),
                                year_from_time)

print(cube.coord('year'))